In [ ]:
!pip install catboost
!pip install emoji

     |████████████████████████████████| 65.8MB 64kB/s 
     |████████████████████████████████| 133kB 7.6MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir("/content/drive/MyDrive/sent-analysis")
from feature_extractions_helper import *

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import altair as alt
import nltk
import numpy as np
import torchtext
import scipy
import torch
import torch.nn as nn

from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.dummy import DummyClassifier
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    cross_val_score,
    cross_validate,
    train_test_split,
)
from sklearn.impute import SimpleImputer
from collections import Counter
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, StackingClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score, make_scorer

from xgboost import XGBClassifier
from lightgbm.sklearn import LGBMClassifier
from catboost import CatBoostClassifier

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from wordcloud import WordCloud, STOPWORDS
from scipy.stats import lognorm, loguniform, randint
from collections import Counter

from torch.utils.data import Dataset, DataLoader
from torch import optim
from torchtext.data import Field, LabelField
from torchtext.data import TabularDataset
from torchtext.data import Iterator, BucketIterator
from torchtext.vocab import Vectors


nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
df = pd.read_csv("../data/processed/processed_data_7760.csv", index_col=0)

In [ ]:
train_df, validation_df = train_test_split(df, test_size = 0.3, random_state=123)

In [ ]:
val_df, test_df = train_test_split(validation_df, test_size=0.5)

In [ ]:
print(train_df.shape)
print(val_df.shape)
print(test_df.shape) 

(5432, 33)
(1164, 33)
(1164, 33)


In [ ]:
drop_features = ["created_at", "id", "entities", "is_quote_status", "location", "tags_count"] # I'll add tags_counts later
target = "sentiment"
categorical_features = ["keyword", "city", "province", "country"]
text_feature = "processed_text"

In [ ]:
numeric_list = train_df.select_dtypes(include=np.number).columns.tolist()
numeric_set = set(numeric_list) - {"id"}
# numeric_set.add("possibly_sensitive") # numeric feature
numeric_features = list(numeric_set)
numeric_features

['cap_words_count',
 'avg_pos_words',
 'second_person_pron_count',
 'avg_neg_words',
 'retweet_count',
 'hashtags_count',
 'avg_slang_sent',
 'avg_len_sent',
 'third_person_pron_count',
 'coord_conj_count',
 'favorite_count',
 'future_tense_count',
 'multi_punc_count',
 'commas_count',
 'first_preson_pron_count',
 'sentence_count',
 'slang_acronym_count',
 'past_tense_count',
 'avg_len_tokens']

In [ ]:
keep_columns = ["sentiment", "processed_text"] + numeric_features
keep_columns

['sentiment',
 'processed_text',
 'cap_words_count',
 'avg_pos_words',
 'second_person_pron_count',
 'avg_neg_words',
 'retweet_count',
 'hashtags_count',
 'avg_slang_sent',
 'avg_len_sent',
 'third_person_pron_count',
 'coord_conj_count',
 'favorite_count',
 'future_tense_count',
 'multi_punc_count',
 'commas_count',
 'first_preson_pron_count',
 'sentence_count',
 'slang_acronym_count',
 'past_tense_count',
 'avg_len_tokens']

In [ ]:
import os
save_path = "../data/split/"
if not os.path.exists(save_path):
    os.mkdir(save_path)

In [ ]:
train_df[keep_columns].to_csv("../data/split/train.tsv", sep='\t', index=False)
val_df[keep_columns].to_csv("../data/split/val.tsv", sep='\t', index=False)
test_df[keep_columns].to_csv("../data/split/test.tsv", sep='\t', index=False)

In [ ]:
train_df = pd.read_csv("../data/split/train.tsv", delimiter="\t")
val_df = pd.read_csv("../data/split/val.tsv", delimiter="\t")
test_df = pd.read_csv("../data/split/test.tsv", delimiter="\t")

In [ ]:
TEXT = Field(sequential=True, tokenize=word_tokenize, lower=True, stop_words=STOPWORDS)
NUM_FEATS = Field(sequential=False, use_vocab=False, unk_token=False, dtype=torch.float)
LABEL = Field(sequential=False, unk_token=None)

In [ ]:
train_set, val_set, test_set = TabularDataset.splits(path="../data/split/",
                                         train="train.tsv", validation="val.tsv", test="test.tsv",
                                         format="tsv",
                                         skip_header=True,
                                         fields=[('label', LABEL),
                                                 ("content", TEXT),
                                                 ('favorite_count', NUM_FEATS),
                                                 ('third_person_pron_count', NUM_FEATS),
                                                 ('past_tense_count', NUM_FEATS),
                                                 ('retweet_count', NUM_FEATS),
                                                 ('commas_count', NUM_FEATS),
                                                 ('avg_slang_sent', NUM_FEATS),
                                                 ('coord_conj_count', NUM_FEATS),
                                                 ('future_tense_count', NUM_FEATS),
                                                 ('multi_punc_count', NUM_FEATS),
                                                 ('second_person_pron_count', NUM_FEATS),
                                                 ('hashtags_count', NUM_FEATS),
                                                 ('slang_acronym_count', NUM_FEATS),
                                                 ('cap_words_count', NUM_FEATS),
                                                 ('avg_len_sent', NUM_FEATS),
                                                 ('first_preson_pron_count', NUM_FEATS),
                                                 ('sentence_count', NUM_FEATS),
                                                 ('avg_neg_words', NUM_FEATS),
                                                 ('avg_pos_words', NUM_FEATS),
                                                 ('avg_len_tokens', NUM_FEATS),
                                                 ])

In [ ]:
train_set.fields

{'avg_len_sent': <torchtext.data.field.Field at 0x7f60793ec860>,
 'avg_len_tokens': <torchtext.data.field.Field at 0x7f60793ec860>,
 'avg_neg_words': <torchtext.data.field.Field at 0x7f60793ec860>,
 'avg_pos_words': <torchtext.data.field.Field at 0x7f60793ec860>,
 'avg_slang_sent': <torchtext.data.field.Field at 0x7f60793ec860>,
 'cap_words_count': <torchtext.data.field.Field at 0x7f60793ec860>,
 'commas_count': <torchtext.data.field.Field at 0x7f60793ec860>,
 'content': <torchtext.data.field.Field at 0x7f60793ec7f0>,
 'coord_conj_count': <torchtext.data.field.Field at 0x7f60793ec860>,
 'favorite_count': <torchtext.data.field.Field at 0x7f60793ec860>,
 'first_preson_pron_count': <torchtext.data.field.Field at 0x7f60793ec860>,
 'future_tense_count': <torchtext.data.field.Field at 0x7f60793ec860>,
 'hashtags_count': <torchtext.data.field.Field at 0x7f60793ec860>,
 'label': <torchtext.data.field.Field at 0x7f60793eca20>,
 'multi_punc_count': <torchtext.data.field.Field at 0x7f60793ec860>,

In [ ]:
TEXT.build_vocab(train_set, min_freq=2, vectors=torchtext.vocab.Vectors(name="./glove.twitter.27B/glove.twitter.27B.100d.txt"))
LABEL.build_vocab(train_set)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
train_iter, val_iter = BucketIterator.splits(
    (train_set, val_set),
    batch_sizes=(32,32),
    sort_key=lambda x: len(x.content), 
    sort=True,
    sort_within_batch=True,
    device=device
)

test_iter = Iterator(
    dataset = test_set, # we pass in the datasets we want the iterator to draw data from
    sort = False, # don't sort the examples
    batch_size=32, # batch size
    sort_key=None, # no sorting
    shuffle=False, # don't shuffle the examples
    sort_within_batch=False, # no sorting
    device=device, # cpu vs cuda
    train=False # this is not a train set
)

In [ ]:
for batch in train_iter:
    print(batch.content)
    print(batch.content.shape)
    print(batch.label.shape)
    break

tensor([[  60,   73,    4,   64,  104,    0,  243, 2269, 1284,    3,  239,    8,
            6,    2, 3596, 2282,    0,  274, 2412, 2403, 1472,    6,    8,  386,
          700,  700,  619,    8,  856, 3236, 1778,   30],
        [ 880,    2,    2,   73, 1812,    0,    3, 1315,    2,    0,  164,    2,
          559,    0,    0,    6,    0,  266,    6,    2,    8,   75,    0,   88,
            1,    1,    1,    1,    1,    1,    1,    1],
        [2151,   30,    0,    2,    6,    0,    2,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1]])
torch.Size([3, 32])
torch.Size([32])


In [ ]:
VOCAB_SIZE = len(TEXT.vocab.stoi)
EMBEDDING_SIZE = TEXT.vocab.vectors.shape[1]
NUM_CLASSES = 3
MAX_EPOCHS = 10

In [ ]:
glove = TEXT.vocab.vectors

In [ ]:
import pdb
class SeqModel(nn.Module):
  
  def __init__(self, embedding_size=200, vocab_size=300, output_size=5, hidden_size=200, 
               num_layers=2, nonlin="tanh", dropout_rate=0.7, mode=0, unit="lstm", more_features=False):
              # add glove param here? ^^

    super(SeqModel, self).__init__()
    self.mode = mode
    self.unit = unit
    self.more_features = more_features

    self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_size).from_pretrained(glove, freeze=True)
    # self.embedding.weight.data.normal_(0.0,0.05) # mean=0.0, mu=0.05

    
    if mode == 0:
        if unit == "lstm":
            self.lstm_rnn = nn.LSTM(input_size=embedding_size, hidden_size=hidden_size, num_layers=num_layers)
        elif unit == "gru":
            self.gru_rnn = nn.GRU(input_size=embedding_size ,hidden_size=hidden_size, num_layers=num_layers)
        else:
            #baseline: unidirectional rnn
            self.rnn = nn.RNN(input_size=embedding_size, hidden_size=hidden_size, num_layers=num_layers, nonlinearity=nonlin)
        if more_features:
            self.linear_layer = nn.Linear(hidden_size + len(numeric_features), output_size)
        else:
            self.linear_layer = nn.Linear(hidden_size, output_size)

    #model with dropout:
    if mode == 1:
        if unit == "lstm":
            self.lstm_rnn = nn.LSTM(input_size=embedding_size, hidden_size=hidden_size, num_layers=num_layers, dropout=dropout_rate, )
        elif unit == "gru":
            self.gru_rnn = nn.GRU(input_size=embedding_size, hidden_size=hidden_size, num_layers=num_layers, dropout=dropout_rate)
        else:
            self.rnn = nn.RNN(input_size=embedding_size, hidden_size=hidden_size, num_layers=num_layers, nonlinearity=nonlin, dropout=dropout_rate)

        if more_features:
            self.linear_layer = nn.Linear(hidden_size + len(numeric_features), output_size)
        else:
            self.linear_layer = nn.Linear(hidden_size, output_size)
    
    #Bidirectional model
    if mode == 2:
        if unit == "lstm":
            self.lstm_rnn = nn.LSTM(input_size=embedding_size, hidden_size=hidden_size, num_layers=num_layers, dropout=dropout_rate, bidirectional=True)
        elif unit == "gru":
            self.gru_rnn = nn.GRU(input_size=embedding_size, hidden_size=hidden_size, num_layers=num_layers, dropout=dropout_rate, bidirectional=True)
        else:
            self.rnn = nn.RNN(input_size=embedding_size, hidden_size=hidden_size, num_layers=num_layers, nonlinearity=nonlin, dropout=dropout_rate, bidirectional=True)

        if more_features:
            self.linear_layer = nn.Linear(hidden_size * 2 + len(numeric_features), output_size)
        else:
            self.linear_layer = nn.Linear(hidden_size * 2, output_size)

    self.activation_fn = nonlin
    self.softmax_layer = nn.LogSoftmax(dim=1)
  
  def forward(self, x, x_concat=None):
      #permute x?
      out = self.embedding(x)
      if self.unit == "lstm":
          out, (h_state, c_state) = self.lstm_rnn(out)
      elif self.unit == "gru":
          out, h_state = self.gru_rnn(out)
      else:
          out, h_state = self.rnn(out)
      out = out[-1]
      if self.more_features:
        out = torch.cat((out, x_concat.permute(1,0)), dim=1)
      out = self.linear_layer(out)
      out = self.softmax_layer(out)
      return out

In [1]:
# Attribution: UBC MDS-CL supervised learning II lab code from Peter Sullivan
def train(loader, model, criterion, optimizer, device):
    total_loss = 0.0
    # iterate throught the data loader
    num_sample = 0
    for batch in loader:
        # load the current batch
        batch_input = batch.content
        batch_output = batch.label
        batch_numeric = torch.stack([batch.retweet_count,
                         batch.coord_conj_count,
                         batch.first_preson_pron_count,
                         batch.commas_count,
                         batch.slang_acronym_count,
                         batch.avg_neg_words,
                         batch.avg_len_sent,
                         batch.future_tense_count,
                         batch.hashtags_count,
                         batch.cap_words_count,
                         batch.sentence_count,
                         batch.avg_pos_words,
                         batch.avg_len_tokens,
                         batch.past_tense_count,
                         batch.multi_punc_count,
                         batch.avg_slang_sent,
                         batch.third_person_pron_count,
                         batch.favorite_count,
                         batch.second_person_pron_count])
        batch_input = batch_input.to(device)
        batch_output = batch_output.to(device)
        batch_numeric = batch_numeric.to(device)
        # forward propagation
        # pass the data through the model
        model_outputs = model(batch_input, batch_numeric)
        # compute the loss
        cur_loss = criterion(model_outputs, batch_output)
        total_loss += cur_loss.cpu().item()

        # backward propagation (compute the gradients and update the model)
        # clear the buffer
        optimizer.zero_grad()
        # compute the gradients
        cur_loss.backward()
        # update the weights
        optimizer.step()

        num_sample += batch_output.shape[0]

    return total_loss/num_sample

# evaluation logic based on classification accuracy
def evaluate(loader, model, criterion, device):
    all_pred=[]
    all_label = []
    with torch.no_grad(): # impacts the autograd engine and deactivate it. reduces memory usage and speeds up computation
        for batch in loader:
             # load the current batch
            batch_input = batch.content
            batch_output = batch.label
            batch_numeric = torch.stack([batch.retweet_count,
                  batch.coord_conj_count,
                  batch.first_preson_pron_count,
                  batch.commas_count,
                  batch.slang_acronym_count,
                  batch.avg_neg_words,
                  batch.avg_len_sent,
                  batch.future_tense_count,
                  batch.hashtags_count,
                  batch.cap_words_count,
                  batch.sentence_count,
                  batch.avg_pos_words,
                  batch.avg_len_tokens,
                  batch.past_tense_count,
                  batch.multi_punc_count,
                  batch.avg_slang_sent,
                  batch.third_person_pron_count,
                  batch.favorite_count,
                  batch.second_person_pron_count])
            
            batch_input = batch_input.to(device)
            batch_numeric = batch_numeric.to(device)
            # forward propagation
            # pass the data through the model
            model_outputs = model(batch_input, batch_numeric)
            # identify the predicted class for each example in the batch
            probabilities, predicted = torch.max(model_outputs.cpu().data, 1)
            # put all the true labels and predictions to two lists
            all_pred.extend(predicted)
            all_label.extend(batch_output.cpu())
            
    accuracy = accuracy_score(all_label, all_pred)
    f1score = f1_score(all_label, all_pred, average='micro') 
    return accuracy, f1score

In [ ]:
# Attribution: UBC MDS-CL supervised learning II lab code from Peter Sullivan
def random_search(num_iter, architecture):    
    results = []
    for i in range(num_iter):
        #define hyperparameters here
        config = {
            "hidden_size": scipy.stats.randint.rvs(100,500),
            "lr": scipy.stats.loguniform.rvs(10**-4,0.1),
            "l2": scipy.stats.loguniform.rvs(10**-5,10**-2),
            "nonlin" : np.random.choice(["relu","tanh"]),
            "dropout":np.random.choice(np.arange(0.1, 1, 0.1)),
            "num_layers": np.random.choice([1,2]),
            "mode": np.random.choice([0,1,2]),
            "optimizer": np.random.choice(["Adam"]),
            "momentum": np.random.choice(np.arange(0.1, 1, 0.1))
        }

        print("new config")
        print(config)
        model = SeqModel(embedding_size=EMBEDDING_SIZE, 
                          vocab_size=VOCAB_SIZE, 
                          output_size=NUM_CLASSES, 
                          hidden_size=config["hidden_size"], 
                          num_layers=config["num_layers"], 
                          nonlin=config["nonlin"],
                          dropout_rate=config["dropout"],
                          mode=config["mode"],
                          unit=architecture,
                          more_features=True)
        
        model = model.to(device)
        criterion = nn.NLLLoss()
        if config["optimizer"] == "Adam":
          optimizer = torch.optim.Adam(model.parameters(), lr=config["lr"], weight_decay=config["l2"])
        else:
          optimizer = torch.optim.SGD(model.parameters(), lr=config["lr"], weight_decay=config["l2"], momentum=config["momentum"])

    
        max_val = 0
        best_epoch = 0
        for epoch in range(MAX_EPOCHS):
        # train the model for one pass over the data
            train_loss = train(train_iter, model, criterion, optimizer, device)  
        # compute the training accuracy
            train_acc, train_f1 = evaluate(train_iter, model, criterion, device)
        # compute the validation accuracy
            vac_acc, val_f1 = evaluate(val_iter, model, criterion, device)
            if val_f1 > max_val:
                max_val = val_f1
                best_epoch = epoch + 1
                best_model = model
        # print the loss for every epoch
            print('Epoch [{}/{}], Loss: {:.4f}, Training F1-score: {:.4f}, Validation F1-score: {:.4f}'.format(epoch+1, MAX_EPOCHS, train_loss, train_f1, val_f1))
        results.append((max_val,best_epoch,config))
        print("Best validation score for iterations #{}: {}".format(i,max_val))
    return results, best_model

In [ ]:
units = ["rnn", "gru", "lstm"]
for unit in units:
    print(f"************* Architecture: {unit} *************")
    random_search(20, unit)

************* Architecture: rnn *************
new config
{'hidden_size': 476, 'lr': 0.01678726497584567, 'l2': 0.0005582090037501318, 'nonlin': 'tanh', 'dropout': 0.4, 'num_layers': 2, 'mode': 0, 'optimizer': 'Adam', 'momentum': 0.5}
Epoch [1/10], Loss: 0.1570, Training F1-score: 0.5313, Validation F1-score: 0.5249
Epoch [2/10], Loss: 0.3463, Training F1-score: 0.4608, Validation F1-score: 0.4553
Epoch [3/10], Loss: 0.2238, Training F1-score: 0.5543, Validation F1-score: 0.5301
Epoch [4/10], Loss: 0.3367, Training F1-score: 0.4753, Validation F1-score: 0.4691
Epoch [5/10], Loss: 0.2109, Training F1-score: 0.5394, Validation F1-score: 0.5077
Epoch [6/10], Loss: 0.3940, Training F1-score: 0.5747, Validation F1-score: 0.5404
Epoch [7/10], Loss: 0.2223, Training F1-score: 0.6539, Validation F1-score: 0.6649
Epoch [8/10], Loss: 0.2009, Training F1-score: 0.6272, Validation F1-score: 0.6392
Epoch [9/10], Loss: 0.2388, Training F1-score: 0.6484, Validation F1-score: 0.6598
Epoch [10/10], Loss

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:61: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


Epoch [1/10], Loss: 0.0401, Training F1-score: 0.7180, Validation F1-score: 0.7328
Epoch [2/10], Loss: 0.0280, Training F1-score: 0.7268, Validation F1-score: 0.7199
Epoch [3/10], Loss: 0.0376, Training F1-score: 0.7332, Validation F1-score: 0.7320
Epoch [4/10], Loss: 0.0344, Training F1-score: 0.7255, Validation F1-score: 0.7259
Epoch [5/10], Loss: 0.0470, Training F1-score: 0.7818, Validation F1-score: 0.7680
Epoch [6/10], Loss: 0.0515, Training F1-score: 0.7927, Validation F1-score: 0.7560
Epoch [7/10], Loss: 0.0437, Training F1-score: 0.7909, Validation F1-score: 0.7431
Epoch [8/10], Loss: 0.0388, Training F1-score: 0.7758, Validation F1-score: 0.7320
Epoch [9/10], Loss: 0.0372, Training F1-score: 0.8036, Validation F1-score: 0.7036
Epoch [10/10], Loss: 0.0344, Training F1-score: 0.8212, Validation F1-score: 0.7062
Best validation score for iterations #3: 0.7680412371134021
new config
{'hidden_size': 194, 'lr': 0.00035050251835192306, 'l2': 0.0002553507017480511, 'nonlin': 'tanh', 

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:61: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


Epoch [1/10], Loss: 0.0265, Training F1-score: 0.7027, Validation F1-score: 0.6976
Epoch [2/10], Loss: 0.0270, Training F1-score: 0.7397, Validation F1-score: 0.7397
Epoch [3/10], Loss: 0.0249, Training F1-score: 0.7634, Validation F1-score: 0.7491
Epoch [4/10], Loss: 0.0234, Training F1-score: 0.7691, Validation F1-score: 0.7577
Epoch [5/10], Loss: 0.0222, Training F1-score: 0.7752, Validation F1-score: 0.7517
Epoch [6/10], Loss: 0.0228, Training F1-score: 0.7798, Validation F1-score: 0.7448
Epoch [7/10], Loss: 0.0209, Training F1-score: 0.7765, Validation F1-score: 0.7569
Epoch [8/10], Loss: 0.0215, Training F1-score: 0.7796, Validation F1-score: 0.7569
Epoch [9/10], Loss: 0.0191, Training F1-score: 0.7817, Validation F1-score: 0.7466
Epoch [10/10], Loss: 0.0199, Training F1-score: 0.7881, Validation F1-score: 0.7259
Best validation score for iterations #4: 0.7577319587628866
new config
{'hidden_size': 157, 'lr': 0.08096625285913045, 'l2': 2.1391914914709266e-05, 'nonlin': 'relu', 'd

...

Epoch [1/10], Loss: 0.6762, Training F1-score: 0.5996, Validation F1-score: 0.6082
Epoch [2/10], Loss: 1.2952, Training F1-score: 0.6771, Validation F1-score: 0.6813
Epoch [3/10], Loss: 0.3363, Training F1-score: 0.6607, Validation F1-score: 0.6624
Epoch [4/10], Loss: 2.3181, Training F1-score: 0.7224, Validation F1-score: 0.6864
Epoch [5/10], Loss: 1.0739, Training F1-score: 0.6900, Validation F1-score: 0.6924
Epoch [6/10], Loss: 1.8884, Training F1-score: 0.6951, Validation F1-score: 0.7139
Epoch [7/10], Loss: 1.0974, Training F1-score: 0.7049, Validation F1-score: 0.6667
Epoch [8/10], Loss: 1.6857, Training F1-score: 0.6944, Validation F1-score: 0.6692
Epoch [9/10], Loss: 0.9730, Training F1-score: 0.6891, Validation F1-score: 0.7002
Epoch [10/10], Loss: 0.8919, Training F1-score: 0.7150, Validation F1-score: 0.6942
Best validation score for iterations #4: 0.7139175257731959
new config
{'hidden_size': 226, 'lr': 0.0010273193521579736, 'l2': 0.0008963885814396396, 'nonlin': 'tanh', '

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:61: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.30000000000000004 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


Epoch [1/10], Loss: 0.4562, Training F1-score: 0.6366, Validation F1-score: 0.6546
Epoch [2/10], Loss: 1.2118, Training F1-score: 0.5523, Validation F1-score: 0.5395
Epoch [3/10], Loss: 1.5417, Training F1-score: 0.6854, Validation F1-score: 0.7045
Epoch [4/10], Loss: 0.7774, Training F1-score: 0.6675, Validation F1-score: 0.6813
Epoch [5/10], Loss: 1.3709, Training F1-score: 0.6850, Validation F1-score: 0.7019
Epoch [6/10], Loss: 0.8392, Training F1-score: 0.6697, Validation F1-score: 0.6821
Epoch [7/10], Loss: 1.2670, Training F1-score: 0.6740, Validation F1-score: 0.6890
Epoch [8/10], Loss: 0.6988, Training F1-score: 0.6762, Validation F1-score: 0.6916
Epoch [9/10], Loss: 1.2232, Training F1-score: 0.6355, Validation F1-score: 0.6564
Epoch [10/10], Loss: 0.5537, Training F1-score: 0.6832, Validation F1-score: 0.7019
Best validation score for iterations #15: 0.7044673539518901
new config
{'hidden_size': 353, 'lr': 0.016707262110165218, 'l2': 3.576789060071812e-05, 'nonlin': 'relu', '

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:61: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


Epoch [1/10], Loss: 0.2061, Training F1-score: 0.6499, Validation F1-score: 0.6581
Epoch [2/10], Loss: 0.6580, Training F1-score: 0.6907, Validation F1-score: 0.7079
Epoch [3/10], Loss: 0.3268, Training F1-score: 0.6565, Validation F1-score: 0.6538
Epoch [4/10], Loss: 0.3218, Training F1-score: 0.6592, Validation F1-score: 0.6469
Epoch [5/10], Loss: 0.3339, Training F1-score: 0.6445, Validation F1-score: 0.6177
Epoch [6/10], Loss: 0.3044, Training F1-score: 0.6723, Validation F1-score: 0.6581
Epoch [7/10], Loss: 0.3120, Training F1-score: 0.6414, Validation F1-score: 0.6289
Epoch [8/10], Loss: 0.2921, Training F1-score: 0.6602, Validation F1-score: 0.6581
Epoch [9/10], Loss: 0.3837, Training F1-score: 0.6322, Validation F1-score: 0.6134
Epoch [10/10], Loss: 0.3693, Training F1-score: 0.6880, Validation F1-score: 0.6993
Best validation score for iterations #0: 0.7079037800687286
new config
{'hidden_size': 354, 'lr': 0.00016522149282424393, 'l2': 0.007037724945218993, 'nonlin': 'tanh', '

In [ ]:
MAX_EPOCHS = 20
best_config = {'hidden_size': 302, 
               'lr': 0.00010769630091763721, 
               'l2': 2.5888680371842294e-05, 
               'nonlin': 'tanh', 
               'dropout': 0.1, 
               'num_layers': 2, 
               'mode': 0, 
               'optimizer': 'Adam', 
               'momentum': 0.1
               }

best_model = SeqModel(embedding_size=EMBEDDING_SIZE, 
                  vocab_size=VOCAB_SIZE, 
                  output_size=NUM_CLASSES, 
                  hidden_size=best_config["hidden_size"], 
                  num_layers=best_config["num_layers"], 
                  nonlin=best_config["nonlin"],
                  dropout_rate=best_config["dropout"],
                  mode=best_config["mode"],
                  unit="gru",
                  more_features=False)
best_model = best_model.to(device)
criterion = nn.NLLLoss()
if best_config["optimizer"] == "Adam":
  optimizer = torch.optim.Adam(best_model.parameters(), lr=best_config["lr"], weight_decay=best_config["l2"])
else:
  optimizer = torch.optim.Adam(best_model.parameters(), lr=best_config["lr"], weight_decay=best_config["l2"])


max_val = 0
best_epoch = 0
for epoch in range(MAX_EPOCHS):
    # train the model for one pass over the data
    train_loss = train(train_iter, best_model, criterion, optimizer, device)  
    # compute the training accuracy
    train_acc, train_f1 = evaluate(train_iter, best_model, criterion, device)
    # compute the validation accuracy
    vac_acc, val_f1 = evaluate(val_iter, best_model, criterion, device)
    if val_f1 > max_val:
        max_val = val_f1
        best_epoch = epoch + 1
    # print the loss for every epoch
    print('Epoch [{}/{}], Loss: {:.4f}, Training F1-score: {:.4f}, Validation F1-score: {:.4f}'.format(epoch+1, MAX_EPOCHS, train_loss, train_f1, val_f1))

print("Best validation score for iterations #{}: {}".format(i,max_val))


Epoch [1/20], Loss: 0.0256, Training F1-score: 0.6966, Validation F1-score: 0.7088
Epoch [2/20], Loss: 0.0210, Training F1-score: 0.7515, Validation F1-score: 0.7715
Epoch [3/20], Loss: 0.0192, Training F1-score: 0.7651, Validation F1-score: 0.7784
Epoch [4/20], Loss: 0.0186, Training F1-score: 0.7714, Validation F1-score: 0.7861
Epoch [5/20], Loss: 0.0182, Training F1-score: 0.7771, Validation F1-score: 0.7861
Epoch [6/20], Loss: 0.0178, Training F1-score: 0.7830, Validation F1-score: 0.7852
Epoch [7/20], Loss: 0.0175, Training F1-score: 0.7852, Validation F1-score: 0.7835
Epoch [8/20], Loss: 0.0173, Training F1-score: 0.7909, Validation F1-score: 0.7852
Epoch [9/20], Loss: 0.0170, Training F1-score: 0.7949, Validation F1-score: 0.7809
Epoch [10/20], Loss: 0.0168, Training F1-score: 0.8006, Validation F1-score: 0.7818
Epoch [11/20], Loss: 0.0165, Training F1-score: 0.8017, Validation F1-score: 0.7835
Epoch [12/20], Loss: 0.0163, Training F1-score: 0.8041, Validation F1-score: 0.7784
E

In [ ]:
torch.save(best_model.state_dict,"./model_deploy")